In [ ]:
# ===============================================
# FULL VIDEO → FRAME EXTRACTION + LICENSE PLATE BLUR PIPELINE
# Steps:
#   1. Upload video(s)
#   2. Extract 1 frame every 15 frames
#   3. Save frames to Google Drive
#   4. Detect & blur license plates using YOLOv11 (for privacy)
#   5. Save blurred frames to another Drive folder (Name of Folder can be changed)
#   6. Auto-delete uploaded videos from Colab
# ===============================================

# Step 1 — Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2 — Install dependencies
!pip install --quiet ultralytics opencv-python-headless tqdm huggingface_hub

import os
import cv2
import numpy as np
from tqdm import tqdm
from google.colab import files
from ultralytics import YOLO
from huggingface_hub import hf_hub_download

# Step 3 — Upload video(s) from local computer
print("Please upload your video file(s)...")
uploaded = files.upload()  # user selects videos from computer
video_files = list(uploaded.keys())

# Step 4 — Define folders in Google Drive
drive_frames_folder = '/content/drive/MyDrive/Frames'
drive_blurred_folder = '/content/drive/MyDrive/Extracted_and_Blurred'
os.makedirs(drive_frames_folder, exist_ok=True)
os.makedirs(drive_blurred_folder, exist_ok=True)

# Step 4.1 — Set a custom prefix for all frame filenames
custom_prefix = "Location_4_"  # change this prefix as needed

# Step 5 — Extract frames from each uploaded video
print("\n Extracting frames...")
for video_filename in video_files:
    print(f" Processing video: {video_filename}")
    cap = cv2.VideoCapture(video_filename)

    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Save every 15th frame
        if frame_count % 15 == 0:
            frame_filename = os.path.join(
                drive_frames_folder,
                f'{custom_prefix}{os.path.splitext(video_filename)[0]}_frame_{saved_count:05d}.jpg'
            )
            cv2.imwrite(frame_filename, frame)
            saved_count += 1
        frame_count += 1

    cap.release()
    print(f"Extracted {saved_count} frames from {video_filename}")

    # Delete the uploaded video from Colab (to save space)
    os.remove(video_filename)
    print(f" Deleted temporary file: {video_filename}")

print(f"\n All frames saved in: {drive_frames_folder}")

# Step 6 — Download and load YOLOv11 model from Hugging Face
print("\n Downloading YOLOv11 license plate detection model...")
repo_id = "morsetechlab/yolov11-license-plate-detection"
model_path = hf_hub_download(repo_id=repo_id, filename="license-plate-finetune-v1x.pt")
yolo_plate = YOLO(model_path)
print("Model loaded and ready!")

# Step 7 — Helper function to blur detected regions
def blur_boxes(img, boxes, ksize=(51,51), sigma=30):
    """Apply Gaussian blur to detected boxes."""
    h, w = img.shape[:2]
    for (x1, y1, x2, y2) in boxes:
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        if x2 <= x1 or y2 <= y1:
            continue
        roi = img[y1:y2, x1:x2]
        if roi.size > 0:
            blurred = cv2.GaussianBlur(roi, ksize, sigma)
            img[y1:y2, x1:x2] = blurred
    return img

# Step 8 — Run detection and blur on extracted frames
frame_files = sorted([
    f for f in os.listdir(drive_frames_folder)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])
print(f"\n Found {len(frame_files)} frames to process for blurring...")

conf_thresh = 0.35  # Detection confidence threshold

for fname in tqdm(frame_files, desc="Blurring license plates"):
    in_path  = os.path.join(drive_frames_folder, fname)
    out_path = os.path.join(drive_blurred_folder, fname)

    img = cv2.imread(in_path)
    if img is None:
        print(f" Skipped unreadable file: {fname}")
        continue

    # YOLO inference
    results = yolo_plate.predict(img, verbose=False)[0]

    boxes = []
    for box, conf, cls in zip(results.boxes.xyxy, results.boxes.conf, results.boxes.cls):
        if float(conf) >= conf_thresh:
            boxes.append(box.cpu().numpy())

    # Apply blur if plates found
    if boxes:
        img = blur_boxes(img, boxes)
    # Save (blurred or original)
    cv2.imwrite(out_path, img)

print(f"\n Done! Blurred frames saved in: {drive_blurred_folder}")
